# Testing sequencers for possible use in KAPA software

### Preamble

In [1]:
import numpy as np
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
from PyQt5 import QtWidgets as qtw, QtGui as qtg, QtCore as qtc

In [2]:
# %gui qt

In [3]:
# class LightWindow(qtw.QWidget):
#     def __init__(self):
#         super().__init__()
#         self.setLayout(qtw.QHBoxLayout())
#         self.layout().setAlignment(qtc.Qt.AlignJustify)
#         self.resize(500, 500)
#         voltage_slider = qtw.QSlider(qtc.Qt.Vertical)
#         self.layout().addWidget(voltage_slider)
#         self.show()

In [4]:
# # Example application
# app = qtw.QApplication([])
# lw = LightWindow()
# app.exec()

### Package: python-statemachine
Sequencer based on the concept of a finite state machine. Not specifically built for use with PyEpics or EPICS interface.
<a href=https://python-statemachine.readthedocs.io/en/latest/index.html>Documentation here</a>

In [2]:
from statemachine import StateMachine, State

#### Test case: Light Switch
To test these sequencer packages, I will implement a test case based on a light bulb with varying voltage.

In [3]:
# Subclass state machine
class LightMachine(StateMachine):
    off = State('Off', initial=True)
    on = State('On')

    turn_off = on.to(off)
    turn_on = off.to(on)
    
    def on_turn_on(self):
        print("Light on ", end='\r')
    def on_turn_off(self):
        print("Light off", end='\r')

    def transition(self, voltage):
        if self.is_on and voltage<5.0:
            self.turn_off()
        elif self.is_off and voltage>5.0:
            self.turn_on()

In [4]:
light = LightMachine()
count = 1000
inc = 0.01
voltage = 0
# varies voltage
for _ in range(count):
    inc = -inc if voltage>=10.0 or voltage<0.0 else inc
    voltage += inc
    light.transition(voltage)
    time.sleep(0.005)

### Package: transitions
Implements a different version of a finite state machine in Python.
<a href=https://github.com/pytransitions/transitions#quickstart>Documentation here</a>

In [5]:
from transitions import Machine

In [6]:
class LightBulb:
    states = ['on', 'off']
    def __init__(self):
        self.machine = Machine(model=self, states=self.states, initial=self.states[0])
        
        self.machine.add_transition(trigger='next', source='*', dest='on', conditions=['enough_voltage'], after='print_state')
        self.machine.add_transition(trigger='next', source='*', dest='off', after='print_state')
    
    @property
    def enough_voltage(self):
        """ Simulates an AC current """
        return (int(time.time()*100) % 10) > 5
    
    def print_state(self):
        if self.state=='on':
            print("light on ", end='\r')
        else:
            print("light off", end='\r')

In [7]:
light = LightBulb()
for i in range(5000):
    light.next()
    time.sleep(0.01)

#### Test case: Parity Checker
Another simple test case is a binary parity checker - whether there are an even number of zeros or ones in a binary number. For this test case we will pass the number in as a string for ease of programming and checking for an even number of ones.

In [48]:
test_even = "100101011101001" # even
test_odd = "01101011011001111" # odd
test_invalid = "1011023011001" # no numbers allowed > 1

##### With statemachine:

In [65]:
class ParityMachine(StateMachine):
    even = State('Even', initial=True)
    odd = State('Odd')
    invalid = State("Invalid")
    
    to_even = odd.to(even)
    to_odd = even.to(odd)
    even_invalid = even.to(invalid)
    odd_invalid = odd.to(invalid)
    reset_inv = invalid.to(even)
    
    def result(self):
        if not self.is_invalid:
            return self.is_even
        else:
            return "Invalid number"
    
    def transition(self, key):
        # Notice we have to resort to just not calling certain transitions
        # if we don't want to individually name them
        if key=='1':
            if self.is_even:
                self.to_odd()
            elif self.is_odd:
                self.to_even()
        elif key!='0':
            if self.is_even:
                self.even_invalid()
            if self.is_odd:
                self.odd_invalid()
    
    def read(self, binary):
        for num in binary:
            self.transition(num)
        
        print(self.current_state.identifier)
        
        if self.is_odd:
            self.to_even()
        if self.is_invalid:
            self.reset_inv()

In [68]:
# Had to work out many bugs, but it works
# MUCH more code than the other one
pm = ParityMachine()
pm.read(test_even)
pm.read(test_odd)
pm.read(test_invalid)

even
odd
invalid


##### With transitions:

In [30]:
class ParityChecker:
    states = ["even", "odd", "invalid"]
    def __init__(self):
        self.machine = Machine(model=self, states=self.states, initial='even', ignore_invalid_triggers=True)
        self.machine.add_transition('next', 'even', 'odd', conditions='one', unless='inv')
        self.machine.add_transition('next', 'odd', 'even', conditions='one', unless='inv')
        self.machine.add_transition('next', '*', 'invalid', conditions='inv')
        self.machine.add_transition('reset', '*', 'even')
    
    def one(self, num):
        return num=='1'
    
    def inv(self, num):
        return num!='1' and num!='0'
    
    def read(self, binary):
        for num in binary:
            self.next(num)
        print(self.state)
        self.reset()

In [37]:
# Worked on like the first or second try
pc = ParityChecker()
pc.read(test_even)
pc.read(test_odd)
pc.read(test_invalid)

even
odd
invalid
